In [ ]:
# Arxiv
# pip install arxiv
# pip install wikipedia

In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


### Wikipedia -Tool 1

In [3]:
# used inbuilt tools in wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

### Arxiv Tool 2

In [4]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [5]:
tools = [wiki, arxiv]

### any web page Tool 3

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [24]:
loader = WebBaseLoader('https://groq.com/pricing/')
docs = loader.load()
documents = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model='gemma2:2b'))
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E85CA76110>, search_kwargs={})

In [25]:
from langchain.tools.retriever import create_retriever_tool
# retriver, name, question
retriever_tool = create_retriever_tool(retriever, 'langsmith_search', 'search any information about langsmith')

In [26]:
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\generativeAi\\langchain\\myenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E8699734C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E85CA76110>, sea

## Running the tools with agents and llms

In [27]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(groq_api_key=groq_api_key, model_name = 'Llama3-8b-8192')

In [28]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
# This is a alraedy exsit prompt
prompt = hub.pull('hwchase17/openai-functions-agent')


### By themselves, language models can't take actions - they just output text. Agents are systems that take a high-level task and use an LLM as a reasoning engine to decide what actions to take and execute those actions.

In [29]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [30]:
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent= agent, tools = tools, verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [22]:
agent_executer.invoke({'input': "what is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'Machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `langsmith_search` with `{'query': 'Machine learning optimization'}`


Wikipedia is a multilingual free online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and using a wiki-based editing system called MediaWiki. Wikipedia is the largest and most-read referenc

{'input': 'what is machine learning',
 'output': 'It seems like the tool call "call_91fw" provided a lot of information related to Wikipedia and LangChain. Since the question was about machine learning, I\'ll try to summarize the relevant information and provide a direct response.\n\nMachine learning is a subset of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data and make predictions or decisions without being explicitly programmed.\n\nThe tool call provided some information about optimization for machine learning, which is a crucial aspect of machine learning. Optimization is the process of finding the best possible solution from a large number of possibilities, and it\'s often used in machine learning to improve the performance of models.\n\nHowever, since the question was not specific to optimization for machine learning, I\'ll provide a more general response. Machine learning is a rapidly growing field with a w

In [23]:
agent_executer.invoke({'input': "what is langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


No good Wikipedia Search Result was found
Invoking: `langsmith_search` with `{'query': 'No good Wikipedia Search Result was found'}`


Wikipedia is a multilingual free online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and using a wiki-based editing system called MediaWiki. Wikipedia is the largest and most-read reference work in history.

Wikipedia | 🦜️🔗 LangChain

Tool conceptual guide
Tool how-to guides
Edit this pageWas this page helpful?PreviousWikidataNextWolfram AlphaRelatedCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2024 LangChain, Inc.

First, you need to install wikipedia python package.
%pip install --upgrade --quiet  wikipedia
from langchain_community.tools import WikipediaQueryRunfrom langchain_community.utilities import WikipediaAPIWrapperAPI Reference:WikipediaQueryRun | W

{'input': 'what is langsmith',
 'output': "It seems like you've provided a lot of information about Wikipedia and LangChain! 😊\n\nAs a helpful assistant, I'll try to summarize the main points for you:\n\n1. Wikipedia is a multilingual online encyclopedia written by a community of volunteers.\n2. LangChain is a tool that provides information on various topics, including Wikipedia.\n3. To use LangChain, you need to install the Wikipedia Python package and run a query.\n\nRegarding your original question, it seems like you were asking about Langsmith, which is a tool that provides information on various topics. However, based on the output you provided, it appears that Langsmith is actually a community-driven platform that provides information on a wide range of topics, including Hunter × Hunter.\n\nIf you have any further questions or would like to know more about Langsmith or Wikipedia, feel free to ask! 😊"}

In [31]:
agent_executer.invoke({'input': "what is groq pricing"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Groq pricing'}`


Page: List of unicorn startup companies
Summary: This is a list of unicorn startup companies:
In finance, a unicorn is a privately held startup company with a current valuation of US$1 billion or more. Notable lists of unicorn companies are maintaine
Invoking: `wikipedia` with `{'query': 'Groq pricing'}`


Page: List of unicorn startup companies
Summary: This is a list of unicorn startup companies:
In finance, a unicorn is a privately held startup company with a current valuation of US$1 billion or more. Notable lists of unicorn companies are maintaine
Invoking: `arxiv` with `{'query': 'Groq pricing'}`
responded: It seems like the previous tool call did not yield the expected result. Since the result was the same as before, I will try again using a different tool.



Published: 2024-10-13
Title: Empowering Dysarthric Speech: Leveraging Advanced LLMs for Accurate Speech Correction and Multi

{'input': 'what is groq pricing',
 'output': "It looks like the tool call yielded some information about Groq pricing! Since the tool call was successful, I can now provide a direct response to the user's original question.\n\nGroq pricing information includes various plans and models for Automatic Speech Recognition (ASR) and Vision models, with different input and output token prices per million tokens. The pricing also includes a minimum of 10 seconds per request for ASR models.\n\nPlease note that the user may want to explore the provided pricing information further to get a better understanding of the costs involved."}